# 通过RF state生成covariant encoding，基于5-qubit code

# RF state更改为2N个qubit组成的sine state
- 仅需更改p(g|h)即可
- 计算entanglement error与worst-case error

# 注：此处计算entanglement fidelity
- 由于仅分析qubit covariant channel，故：
    1. entanglement error = 1-Fent'
    2. worst case error <= 2 * entanglement error

# 1. 需要用到的模块与常用的矩阵

In [5]:
import numpy as np
import random
import time
import matplotlib.pyplot as plt  # 用于作图
from scipy.optimize import curve_fit  # 用于拟合
from scipy.stats import unitary_group  # 生成随机U矩阵


# 通过优化得到测量结果，加载量桨相关模块
import paddle
from paddle_quantum.circuit import UAnsatz  # 3. 创立量子线路
from paddle_quantum.state import vec, vec_random  # 4. 创立量子态
from paddle_quantum.state import density_op, density_op_random, completely_mixed_computational  # 4. 创立量子态
from paddle_quantum.utils import dagger, haar_unitary  # 对paddle.tensor形式取dagger (对numpy.array通过 .conj().T)
from paddle import matmul, trace  # 计算矩阵内积与迹(对tensor)


In [6]:
# 常用矩阵设置

# Bell态 -- np ket
bell_state = np.array([[1,0,0,1]]) / (2**0.5)
bell_state = bell_state.conj().T

# 给出Pauli矩阵
pauli_x = np.array([[0.0, 1.0], [1.0, 0.0]])
pauli_y = np.array([[0.0, -1.0j], [1.0j, 0.0]])
pauli_z = np.array([[1.0, 0.0], [0.0, -1.0]])

# 2. 生成随机SU(2)矩阵

In [7]:
# 2. 生成随机欧拉角表示SU(2) -- U2_random_Euler_mod()
def Rz(alpha):
    # 给出绕Z轴旋转alpha角度np.matrix
    # 输出U：np.array形式
    U = np.array([[0.+0.j, 0.+0.j],[0.+0.j, 0.+0.j]])
    U[0][0] = np.e ** (-1j * alpha/2)
    U[1][1] = np.e ** (1j * alpha/2)
    
    return U

def Ry(alpha):
    # 给出绕Y轴旋转alpha角度np.matrix
    # 输出U：np.array形式
    U = np.array([[0.+0.j, 0.+0.j],[0.+0.j, 0.+0.j]])
    U[0][0] = np.cos(alpha/2)
    U[0][1] = -np.sin(alpha/2)
    U[1][0] = np.sin(alpha/2)
    U[1][1] = np.cos(alpha/2)
    
    return U

def U2_Euler(alpha, beta, gamma):
    # 输出U：np.array形式
    # 对应欧拉角表示：U(alpha, beta, gamma) = Z(gamma) @ Y(beta) @ Z(alpha)
    # 注：alpha, gamma \in [0,2pi], beta \in [0,pi]
    U = Rz(gamma) @ Ry(beta) @ Rz(alpha)
    return U

def U2_random_Euler():
    # 输出U：np.array形式
    # 以均匀几率给出alpha, beta, gamma \in [0, 2pi], 得到最后的U
    alpha = random.uniform(0, 2*np.pi)
    beta = random.uniform(0, np.pi)
    gamma = random.uniform(0, 2*np.pi)
    U = Rz(gamma) @ Ry(beta) @ Rz(alpha)
    return U

def random_sinx(a, b):
    # 生成定义域x \in [a,b]，分布为sin(x)dx的随机x
        # 注：此定义域需保证sin(x) > 0
    # 构造方法：随机生成(x,y)于x属于定义域，y属于[0, 最大值]
        # 对(xi, yi)，若0<yi<f(xi)，则输出xi
    ymax = 1  # sin(x)总小于1
    while 1:
        x = random.uniform(a, b)
        y = random.uniform(0, 1)
        #print(x,y)
        #print('sinx:', np.sin(x))
        if y < np.sin(x):
            break
    return x,y

def U2_random_Euler_mod():
    # 输出U：np.array形式
    # beta分布改为sin(beta)d(beta), alpha, gamma仍然均匀
    alpha = random.uniform(0, 2*np.pi)
    beta, anc = random_sinx(0, np.pi)
    gamma = random.uniform(0, 2*np.pi)
    U = Rz(gamma) @ Ry(beta) @ Rz(alpha)
    
    return U

# 3. 对经过Ug的2N个qubit组成纠缠态，测量得到最终Uh (此部分需要修改)
- random_fromg_geth(Ug) -- random_fromg_geth_largeRF(Ug)
    1. 对N,d，给出via的lambda set以及对应q_lambda -- 此处采用sine state
    2. 计算此RF下p(g|h)
    3. 得到list符合分布p(h|g)
    4. 不需要通过优化，直接通过此分布给出测量结果Uh

In [8]:
# 3.1 对N,d，给出via的lambda set以及对应q_lambda，并归一化 -- 此处采取sine state
# 给出此时RF state系数列表
def q_lambda(N, d):
    # d:SU(d) -- 目前仅可对d=2
    # N = m -- 对SU(2)
    # 输出：两个列表
        # lambda_via：保留此时可行lambda_1
        # q_via：对此lambda_1，对应系数
    lambda_via = []
    q_via = []
    J = N / 2
    if d == 2:
        for lambda_2 in range(1, N // 2 + 1):
            lambda_1 = N - lambda_2
            j = (lambda_1 - lambda_2) / 2
            q = np.sin( np.pi * j  / J ) ** 2 / (2 * j + 1)
            if j != 0:
                lambda_via.append(lambda_1)  # 保留此时lambda_1
                q_via.append(q)

    return lambda_via, q_via

# 归一化
def q_lambda_nor(lambda_via, q_via):
    # 输入未归一化：lambda_via, q_via
    # 给出归一化：lambda_via, q_via
    total_q = 0
    for q in q_via:
        total_q += q
    for i in range(len(q_via)):
        q_via[i] = q_via[i] / total_q
    return lambda_via, q_via

# 给出Ug下测得Uh几率
def pgh_largeRF(N, d, Ug, Uh):
    # d:SU(d) -- 目前仅可对d=2
    # N = m -- 对SU(2)
    # p(h|g) = | \Sum_{lambda_i \in lambda_via} sqrt{q_lambda_i} *  * Tr[(Ug * Uh^{dag, j})]  |^2
        # j = (lambda_i1 - lambda_i2) / 2
        # Tr[U^{1/2}] = 2 cos(\phi/2)
        # Tr[U^{j}] = sin(\phi * (2j + 1)/2) / sin(\phi/2)
    
    # 1. 给出归一化后lambda_via, q_via
    lambda_via, q_via = q_lambda(N, d)
    lambda_via, q_via = q_lambda_nor(lambda_via, q_via)
    
    # 2. 给出\phi
    phi = np.arccos(np.real(np.trace(Ug @ Uh.conj().T)) / 2) * 2  # 取值范围[0, 2*Pi]
    if phi == 0:
        phi = phi + 0.000001
    pghi = 0
    for i in range(len(lambda_via)):
        lambda_i1 = lambda_via[i]
        lambda_i2 = N - lambda_i1
        j = (lambda_i1 - lambda_i2) / 2
        pghi += np.sqrt(q_via[i]) * np.sin(phi * (2*j + 1)/2) / np.sin(phi/2)
    pgh = pghi ** 2
    
    return pgh

- 不需要通过优化，直接通过此分布给出测量结果Uh

In [9]:
# 3.2 由Ug得到测量最终Uh
def random_fromg_geth_largeRF(N, d, Ug):
    # 给定输入Ug，测量几率为p(h|g) = pgh_largeRF(N, d, Ug, Uh)
    # 输出Uh，服从分布p(h|g)
    # 构造方法与random_sinx相同
    pgh_max = pgh_largeRF(N, d, -np.eye(2), -np.eye(2))  # p(g,h)总小于最大值
    while 1:
        Uh = U2_random_Euler_mod()
        #Uh = haar_unitary(1)
        # 转化为SU(2)
        coeUh = np.sqrt(np.linalg.det(Uh))
        Uh = coeUh.conj() * Uh
        #if np.trace(Uh) < 0:
        #    Uh = -Uh
        pgh = random.uniform(0, pgh_max)
        fgh = pgh_largeRF(N, d, Ug, Uh)
        #print('fgh:', fgh)
        #print('pgh:', pgh)
        if pgh < fgh:
            break
    return Uh

# 主体函数：
def From_g_get_hath_largeRF(N, d, Ug):
    Uh = random_fromg_geth_largeRF(N, d, Ug)
    return Uh


# 修改部分到此结束，后续使用From_g_get_hath_largeRF(N, d, Ug)即可

# 4. 关于code部分 -- 与5-qubit code相关

In [10]:
# 第一部分相关函数，与encoding, decoding相关

# 构建函数：2进制转10进制 ---- 用于encoding构建
def TwoToTen(a_two):
    # 输入为2进制字符串: a_two
    # 输出为10进制数字:a_ten
    a_ten = 0
    for i in range(len(a_two)):
        new = int(a_two[len(a_two)-i-1])  # 得到倒数第"i+1"位数字
        a_ten = a_ten + new * (2**i)
    
    return a_ten

# 构建函数：10进制转2进制 ---- 用于decoding结果表示
def TenToTwo(a_ten, n):
    # 输入为10进制数字: a_ten, 需要位数：n
    # 输出为2进制字符串:a_two
    a_two = ''
    while a_ten:
        new = a_ten % 2
        new = str(new)
        a_two += new
        a_ten = a_ten // 2
    if len(a_two) < n:
        for i in range(n - len(a_two)):
            a_two += '0'
    a_two = a_two[::-1]
    
    return a_two

# 构建函数：得到5-qubit code在测量后得到几率与输出态，对应state为4 measure, 5 code, 1 ancilla
def MeasFiveQ(final_state, basis):
    # 输入为circuit输出state (numpy density matrix), 与测量outcome (0对应0000,1对应0001,...15对应1111)
    # 输出为测得几率与余下state的numpy density matrix
        # pr, meas_prob_basis
    meas_prob_basis = np.kron(vec(basis, 4), np.eye(2**6)) @ final_state @ np.kron(vec(basis, 4).conj().T, np.eye(2**6))
    pr = np.real(meas_prob_basis.trace())
    if pr > 0:  # 要求pr不可以为0否则后续会出错
        meas_prob_basis = meas_prob_basis / pr  # 归一化
    
    return pr, meas_prob_basis

# 构建函数：decoding中加入stabilizer measure对应circuit
def stabilizer_meas(cir):
    # 输入为10qubit初始电路cir
    # 初始四个H门
    for i in range(4):
        cir.h(i)
    # 一：对XZZXI
    cir.cnot([0,4])
    cir.cz([0,5])
    cir.cz([0,6])
    cir.cnot([0,7])
    # 二：对IXZZX
    cir.cnot([1,5])
    cir.cz([1,6])
    cir.cz([1,7])
    cir.cnot([1,8])
    # 三：对XIXZZ
    cir.cnot([2,6])
    cir.cz([2,7])
    cir.cz([2,8])
    cir.cnot([2,4])
    # 四：对ZXIXZ
    cir.cnot([3,7])
    cir.cz([3,8])
    cir.cz([3,4])
    cir.cnot([3,5])
    # 最后四个H门
    for i in range(4):
        cir.h(i)
        
    return None

# 构建函数：decoding中根据测量结果，补充对code的修正
def correct_accoring_tomeas(meas_out, outstate):
    # 输入：测量结果，为字符串：meas_out; code \otimes anc对应6 qubit state：outstate -- np.matrix
    # 输出：经过纠正后final_state -- np.matrix
    # 构建对应U_cor
    if meas_out == '0000':
        U_cor = np.eye(2**6)
    elif meas_out == '0001':
        U_cor = np.kron(pauli_x, np.eye(2**5))
        #cir.rx(pi_pad, 0)
    elif meas_out == '1000':
        U_cor = np.kron(np.eye(2**1), np.kron(pauli_x, np.eye(2**4)))
        #cir.rx(pi_pad, 1)
    elif meas_out == '1100':
        U_cor = np.kron(np.eye(2**2), np.kron(pauli_x, np.eye(2**3)))
        #cir.rx(pi_pad, 2)       
    elif meas_out == '0110':
        U_cor = np.kron(np.eye(2**3), np.kron(pauli_x, np.eye(2**2)))
        #cir.rx(pi_pad, 3)
    elif meas_out == '0011':
        U_cor = np.kron(np.eye(2**4), np.kron(pauli_x, np.eye(2**1)))
        #cir.rx(pi_pad, 4)
    elif meas_out == '1010':
        U_cor = np.kron(pauli_z, np.eye(2**5))
        #cir.rz(pi_pad, 0)
    elif meas_out == '0101':
        U_cor = np.kron(np.eye(2**1), np.kron(pauli_z, np.eye(2**4)))
        #cir.rz(pi_pad, 1) 
    elif meas_out == '0010':
        U_cor = np.kron(np.eye(2**2), np.kron(pauli_z, np.eye(2**3)))
        #cir.rz(pi_pad, 2)
    elif meas_out == '1001':
        U_cor = np.kron(np.eye(2**3), np.kron(pauli_z, np.eye(2**2)))
        #cir.rz(pi_pad, 3)
    elif meas_out == '0100':
        U_cor = np.kron(np.eye(2**4), np.kron(pauli_z, np.eye(2**1)))
        #cir.rz(pi_pad, 4)
    elif meas_out == '1011':
        U_cor = np.kron(pauli_y, np.eye(2**5))
        #cir.ry(pi_pad, 0) 
    elif meas_out == '1101':
        U_cor = np.kron(np.eye(2**1), np.kron(pauli_y, np.eye(2**4)))
        #cir.ry(pi_pad, 1)
    elif meas_out == '1110':
        U_cor = np.kron(np.eye(2**2), np.kron(pauli_y, np.eye(2**3)))
        #cir.ry(pi_pad, 2)
    elif meas_out == '1111':
        U_cor = np.kron(np.eye(2**3), np.kron(pauli_y, np.eye(2**2)))
        #cir.ry(pi_pad, 3)
    else:  # meas_out == '0111':
        U_cor = np.kron(np.eye(2**4), np.kron(pauli_y, np.eye(2**1)))
        #cir.ry(pi_pad, 4)
        
    # 得到final_state
    final_state = U_cor @ outstate @ U_cor.conj().T
    
    return final_state


# 5. 关于code部分 -- 得到经过encoding state
- 5-qubit code encoding algorithm
- 经过encoding后得到量子态对应空间：4 meas * 5 code * 1 anc

In [11]:
# Encoding部分参数，直接此处给出，因后续多个函数需要用
# 1.1.1 原encoding为
# |0_L> = 1/4*(00000 + 10010 + 01001 + 10100 + 01010 - 11011 - 00110 - 11000 - 11101 - 00011 - 11110 - 01111 - 10001 - 01100 - 10111 + 00101)
# |1_L> = 1/4*(11111 + 01101 + 10110 + 01011 + 10101 - 00100 - 11001 - 00111 - 00010 - 11100 - 00001 - 10000 - 01110 - 10011 - 01000 + 11010)
    # 注：wiki有误，为保证XXXXX能实现反转，两者正负号应一一对应, 且通过XZZXI验证，正负号关系
    
# 1.1.1.1 构建|0_L>, |1_L>: code_0, code_1
# 生成state, |0_L>，对应np ket
code_0 = np.zeros([2**5, 1])
code_0_list_plus = ['00000', '10010', '01001', '10100', '01010', '00101']
code_0_list_minus = ['11011', '00110', '11000', '11101', '00011', '11110', '01111', '10001', '01100', '10111']
for i in range(len(code_0_list_plus)):
    code_0[TwoToTen(code_0_list_plus[i])] = 1/4
for i in range(len(code_0_list_minus)):
    code_0[TwoToTen(code_0_list_minus[i])] = -1/4
# 对应density matrix
code_0_density = code_0 @ code_0.conj().T
# 生成state, |1_L>，对应np ket
code_1 = np.zeros([2**5, 1])
code_1_list_plus = ['11111', '01101', '10110', '01011', '10101', '11010']
code_1_list_minus = ['00100', '11001', '00111', '00010', '11100', '00001', '10000', '01110', '10011', '01000']
for i in range(len(code_1_list_plus)):
    code_1[TwoToTen(code_1_list_plus[i])] = 1/4
for i in range(len(code_1_list_minus)):
    code_1[TwoToTen(code_1_list_minus[i])] = -1/4
# 对应density matrix
code_1_density = code_1 @ code_1.conj().T
    
# 1.1.1.2 构建isometry: V_Eori
# 生成对应isometry： V_E0 = |0_L><0| + |1_L><1|
V_Eori = code_0 @ vec(0,1) + code_1 @ vec(1,1)

In [12]:
# 得到经过encoding后的state
# 对应covarient encoding: V_Ecov = (Ug)^\otimes 5 V_E0 Ug^dag
# 对应空间： 4 meas * 5 code * 1 anc

def state_afterencoding(Ug):
    # 输入Ug
    # 输出input_total -- paddle.matrix
    
    # 1.1.2.2 构造Ug5，对应code space上矩阵
    Ug5 = Ug
    for i in range(4):
        Ug5 = np.kron(Ug5, Ug)  # 得到(Ug)^\otimes 5
    # 1.1.2.3 构造经过Ug随机encoding isometry
    V_Ecov = Ug5 @ V_Eori @ Ug.conj().T
    # 1.1.3 得到初始态经过covariant encoding后state
    # 此处采用entangement fidelity
    # 选择初始态logical_ini，1 logical \otimes 1 ancilla(在后)
    # 并经过encoding，补充上4个measure qubit，得到input_total为paddle density
    # 顺序为：4 measure \otimes 5 code \otimes 1 ancilla
    # 选择输入态: 1 logical \otimes 1 ancilla(在后)
    logical_ini = bell_state
    # 经过encoding
    # 得到state：input_code, input_total
    input_code = np.kron(V_Ecov, np.eye(2)) @ logical_ini  # 得到code \otimes anc部分，对应np ket
    input_code_density = input_code @ input_code.conj().T  # 输入code \otimes anc部分，对应np的density matrix
    input_total = np.kron(vec(0, 4).conj().T, input_code)  # 加上measure部分
    #input_total = paddle.to_tensor(input_total.conj().T, dtype = 'complex128')  # 为paddle vector
    input_total = input_total * input_total.conj().T  # 对应density matrix
    input_total = paddle.to_tensor(input_total, dtype = 'complex128')  # 为paddle density
    
    return input_total


# 6. 关于code部分 -- 经过noise与decoding后，计算通过Ug encoding以及Uh decoding后的entanglement fidelity

In [13]:
def fidelity_hg(Ug, Uh, input_total, noise):
    # 输入：Ug, Uh -- np.matrix，经过encoding后量子态 -- paddle.matrix，noise模式
        # noise = 1，目前取为第2个code qubit经过completely depolarizing
        # noise = 2，目前取为第2个code qubit经过completely dephasing
    # 输出：Ug encoding以及Uh decoding后，得到的entanglement fideilty
    
    # 1.2 + 1.3 经过noise + 通过Dec_h = Uh @ Dec @ Uh5^dag纠正
    # 1.2 经过noise -- 得到输出state_afternoise -- np.matrix
    # 此处选取为completely depolarizing channel
    # 注：此时的输入为input_total：空间顺序：mea \otimes code \otimes anc ---- paddle.matrix

    # 1.2.1 构建noise channel
    # 经过noise部分
    cir_noise = UAnsatz(10)  # 此code为1qubit -- 4 measure, 5 code, 1 ancilla
    if noise == 1:
        # 选择为第2个code qubit经过completely depolarizing -- 选p = 3/4
        cir_noise.depolarizing(3/4, 5)
    if noise == 2:
        # 选择为第2个code qubit经过completely dephasing -- 选p = 3/4
        cir_noise.phase_flip(1/2, 5)
    #cir_noise.rx(pi_pad, 4)
    state_afternoise = cir_noise.run_density_matrix(input_total).numpy()  # 对应numpy density

    # 1.3 通过Dec_h = Uh @ Dec @ Uh5^dag纠正
    # 核心：给出给定h下最终decoding map，以及performance Per(h)
    # 给出Uh
    # 1.3.1 经过Uh5^dag
    # 1.3.1.2 给出Uh5
    Uh5 = Uh
    for i in range(4):
        Uh5 = np.kron(Uh5, Uh)  # 得到(Uh)^\otimes 5
    # 1.3.1.3 输出state_afterUhdag：空间顺序：mea \otimes code \otimes anc
    # 对经过noise后输出态作用(eye(16) \otimes Uh5 \otimes eye(2))
    state_afterUhdag = np.kron(np.kron(np.eye(16), Uh5.conj().T), np.eye(2)) @ state_afternoise @ np.kron(np.kron(np.eye(16), Uh5.conj().T), np.eye(2)).conj().T
    state_afterUhdag = paddle.to_tensor(state_afterUhdag, dtype = 'complex128')  # 化为paddle density

    # 1.3.2 构造原decoding对应电路 -- cir -- 得到输出态：final_total_density
    # 原decoding D_ori：包括1. stabilizer measure, 2. 根据对4 measure qubit测量结果对余下进行修正, 3. 经过V_Eori^dag 4. 经过 Uh
    # stabilizer measure
    cir = UAnsatz(10)
    stabilizer_meas(cir)
    # 选择输入态
    #state_afterUhdag  # 为10 qubit, paddle density
    # 得到输出state
    final_total_density = cir.run_density_matrix(state_afterUhdag).numpy()  # 对应numpy density

    # 1.3.3 测量mea并修正
    # 注：测量结果有16种情况，每一种有相应修正操作
    # 1.3.3.1 得到每种测量几率与输出state，存于列表
        # prob_meas：32个元素，包括’结果’，对应几率
        # prob_output：16个元素，每个结果对应输出态
    prob_meas = []
    prob_output = []
    for i in range(2**4):
        basis = TenToTwo(i, n=4)
        pr, meas_prob_basis = MeasFiveQ(final_total_density, i)  # 测量10 qubit中前4个
        prob_meas.append(basis+':')
        prob_meas.append(pr)
        prob_output.append(meas_prob_basis)
        #if pr > 0.005:
            #print(basis, pr)

    # 1.3.3.2 对每一种测量结果相应修正 + 1.3.4 对每种测量结果修正后state_aftercor经过V_Eori^dag与Uh
    # 对测量结果求平均得到Per_h
    fidelityh = 0
    for i in range(2**4):
        # 得到state_aftercori, state_afterVdagi, state_afterUhi
        state_aftercori = correct_accoring_tomeas(TenToTwo(i, n=4), prob_output[i])
        state_afterVdagi = np.kron(V_Eori.conj().T, np.eye(2)) @ state_aftercori @ np.kron(V_Eori, np.eye(2))
        state_afterUhi = np.kron(Uh, np.eye(2)) @ state_afterVdagi @ np.kron(Uh.conj().T, np.eye(2))
        # 计算给定h下每种测量结果下Per_(h, mea_out), 对测量结果求平均得到Per_h
        fidelityhi = np.real(bell_state.conj().T @ state_afterUhi @ bell_state)
        fidelityh += fidelityhi * np.real(prob_meas[2*i + 1])
    fidelityh = fidelityh[0][0]
    
    return fidelityh
        
    

# 数值实验
- 对erasure noise

In [ ]:
# 对erasure noise

# 计算平均fidelity与N的关系
# 取ITR_i_hg = 400，循环400次，计算均值方差
    # 1. 随机生成Ug
    # 2. 得到Uh -- 通过From_g_get_hath_largeRF(N, d, Ug)
    # 3. 计算对该Ug下的fidelityh

d = 2
noise = 1  # 取completely depolarizing noise
ITR_i_hg = 400
    
# 最后输出三个列表，对应N_list, f_N_ave_list, f_N_var_list
N_list=[35, 40, 45, 50]  # [20, 30, 40, 50, 60, 70, 80, 90, 100]
f_N_ave_list = []
f_N_var_list = []

# 对每一个N计算fidelity_ave, fidelity_var
for N in N_list:
    print('N:', N)
    time_start = time.time() 

    fidelityh_list =  []  # 将每次的fidelityh存于list中，后续计算均值、方差
    for i_hg in range(ITR_i_hg):
        # 1. 随机生成Ug
        Ug = U2_random_Euler_mod()
        # 2. 得到Uh
        Uh = From_g_get_hath_largeRF(N, d, Ug)
        # 3. 计算对该Ug下的fidelityh
        # 3.1 得到经过(Ug)^\otimes 5 V_E0 Ug^dag后state，input_total -- paddle.matrix
        input_total = state_afterencoding(Ug)
        # 3.2 计算此时得到Uh对应fidelity
        fidelityh = fidelity_hg(Ug, Uh, input_total, noise)  # 此处选depolarizing noise
        fidelityh_list.append(fidelityh)
    # 对ITR_i_hg此求期望与方差
    fidelity_ave = np.mean(fidelityh_list)
    fidelity_var = np.var(fidelityh_list)
    print('fidelity_ave:', fidelity_ave)
    print('fidelity_var:', fidelity_var)
    f_N_ave_list.append(fidelity_ave)
    f_N_var_list.append(fidelity_var)
    
    time_span = time.time() - time_start 
    print('time:', time_span)

In [ ]:
# 拟合并作图 -- 对erasure

# 1. 对以下list作图
#N_list
err_N_ave_list = [2 * (1-i) for i in f_N_ave_list]  # worst-case error <= 2 * entanglement error = 2 * (1 - entanglement fidelity)

# 2. 作图 (原图)
plt.figure(1)
func1, = plt.plot(N_list, err_N_ave_list, 
                  alpha=0.7, marker='', linestyle="-", color='r')
plt.xlabel('n')
plt.ylabel('worst-case error')

plt.legend(handles=[
    func1,
],
    labels=[
        'erasure noise, with composite RF state',
    ], loc='best')

plt.show()

# 3. 拟合
#表现接近于二次反比例函数 err_N_ave (\app coe / N^2)
def func(x, coe):
    return coe / (x**2)
#定义x、y散点坐标
Fitx = N_list
Fitx = np.array(Fitx)
Fity = err_N_ave_list
Fity = np.array(Fity)
 
#非线性最小二乘法拟合
popt, pcov = curve_fit(func, Fitx, Fity)
#获取popt里面是拟合系数
coe = popt[0] 
yvals = func(Fitx, coe) #拟合y值
print('系数coe:', coe)
print('系数pcov:', pcov)
print('拟合error: \n', yvals)
print('原error: \n', err_N_ave_list)
#绘图
Fitxnew = []
Fitynew = []
for i in range(N_list[0], N_list[-1]+1):
    Fitxnew.append(i)
    Fitynew.append(func(i, coe))
#绘图
plot1 = plt.plot(Fitx, Fity, 's',label='original values')
plot2 = plt.plot(Fitxnew, Fitynew, 'r',label='polyfit values')
plt.xlabel('n')
plt.ylabel('worst-case error')
plt.legend(handles=[
    func1,
],
    labels=[
        'erasure noise, with composite RF state',
    ], loc='best')
plt.show()
